# Análise Quantitativa do Trade-off entre Especialização e Generalização em LLMs via Fine-Tuning

**Aluno(s)**: [NOME(s) AQUI]

**Data de Entrega**: 23/06/2025

---

## 1. Objetivo

O objetivo central deste projeto é a avaliação empírica e sistemática do processo de fine-tuning em Modelos de Linguagem de Grande Porte (LLMs). Os alunos irão implementar, treinar e avaliar um LLM para a tarefa de Text-to-SQL. A análise quantificará o ganho de desempenho na tarefa-alvo e, simultaneamente, medirá a degradação de performance em tarefas de conhecimento geral. O projeto exige a implementação de métricas de avaliação customizadas e uma análise crítica dos trade-offs inerentes à especialização de modelos.

## 2. Problema

A especialização de LLMs via fine-tuning é uma técnica proposta para otimizar o desempenho em domínios específicos. Contudo, este processo de otimização focado pode comprometer a robustez do modelo em tarefas que não pertencem ao domínio de treinamento, um fenômeno conhecido como "esquecimento catastrófico" ou "regressão de capacidade". Este trabalho consiste em projetar e executar um pipeline experimental que permita medir com precisão ambas as facetas deste fenômeno: o ganho de especialização e a perda de generalização.

## 3. Materiais e Configuração

* **Modelo Base**: Deve ser utilizado um modelo open-source da classe de 7-8 bilhões de parâmetros em sua versão instruct/chat.
    * *Sugestões*: `meta-llama/Llama-3-8B-Instruct`, `mistralai/Mistral-7B-Instruct-v0.2`.
    * *A versão exata do checkpoint utilizado deve ser documentada para garantir a reprodutibilidade*.
* **Dataset de Fine-Tuning**: `Spider Dataset`, utilizando exclusivamente o `training split`.
* **Dataset de Avaliação de Tarefa**: `Spider development split`.
* **Dataset de Avaliação de Generalização**: `MMLU (Massive Multitask Language Understanding)`.
    * Deve ser criada uma suite de avaliação com **150 questões**, divididas igualmente em 3 categorias: STEM, Humanidades e Ciências Sociais.
* **Framework de Avaliação**: `DeepEval` (versão 0.21.x ou superior).
* **Frameworks de Treinamento**: `Hugging Face TRL`, `Axolotl` ou similar.

### Instalação de Dependências e Setup do Ambiente

Primeiro, vamos instalar as bibliotecas necessárias. É fundamental criar um arquivo `requirements.txt` no seu repositório.

In [5]:
!pip freeze > requirements.txt

In [1]:
# !pip install transformers
# !pip install datasets
# !pip install peft
# !pip install trl
# !pip install bitsandbytes
# !pip install accelerate
# !pip install deepeval
# !pip install pytest
# !pip install triton




In [2]:

import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import pandas as pd

# É obrigatório fixar as sementes (seeds) para garantir a reprodutibilidade.
torch.manual_seed(42)
import random
random.seed(42)
import numpy as np
np.random.seed(42)

/home/alexandre/nlp_metric/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fase 1: Estabelecimento do Baseline de Desempenho

### 1.1. Prompt Engineering

Construir um prompt de few-shot para a tarefa Text-to-SQL. O prompt deve conter 3 exemplos representativos (par linguagem natural -> consulta SQL) extraídos do training split do Spider. Este template de prompt deve ser fixo e utilizado em todas as avaliações de baseline.

In [3]:
# Carregar o dataset Spider para extrair os exemplos
spider_dataset = load_dataset("spider")
train_split = spider_dataset['train']

# Exemplo de como extrair 3 exemplos representativos
example_1 = train_split[0]
example_2 = train_split[1]
example_3 = train_split[2]

# Construa o template de prompt fixo aqui
FEW_SHOT_PROMPT_TEMPLATE = f"""
### INSTRUÇÃO
Dada uma pergunta em linguagem natural, gere a consulta SQL correspondente.

### EXEMPLO 1
Pergunta: {example_1['question']}
SQL: {example_1['query']}

### EXEMPLO 2
Pergunta: {example_2['question']}
SQL: {example_2['query']}

### EXEMPLO 3
Pergunta: {example_3['question']}
SQL: {example_3['query']}

### PERGUNTA ATUAL
Pergunta: {{user_question}}
SQL:"""

print("Template de Prompt Few-Shot Criado.")

Using the latest cached version of the dataset since spider couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'spider' at /home/alexandre/.cache/huggingface/datasets/spider/spider/0.0.0/0c350918f3f29ec754f1181c65cdce76cd6c133c (last modified on Mon Jun 23 14:34:07 2025).


Template de Prompt Few-Shot Criado.


### 1.2. Execução da Avaliação (Baseline)

Submeter o modelo base (não treinado) ao `Spider dev split`, utilizando o template de prompt definido. Registrar a consulta SQL gerada para cada entrada. A métrica a ser reportada nesta fase é a contagem bruta de sucesso/falha.

In [4]:

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",             # Mais eficiente que fp4
    bnb_4bit_compute_dtype=torch.float16,  # Mais leve que bfloat16
    bnb_4bit_use_double_quant=True         # Quantização dupla
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 3/3 [00:41<00:00, 13.79s/it]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm  # Adicione esta linha no início do seu script

# Carregar o split de desenvolvimento do Spider
spider_dev_split = spider_dataset['validation']

# Loop de avaliação com barra de progresso
baseline_results = []

for item in tqdm(spider_dev_split, desc="Gerando SQLs", total=len(spider_dev_split)):
    user_question = item['question']
    ground_truth_query = item['query']
    
    prompt = FEW_SHOT_PROMPT_TEMPLATE.format(user_question=user_question)
    
    # Geração pelo modelo
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = base_model.generate(
        **inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id  
    )
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    baseline_results.append({
        'question': user_question,
        'generated_sql': generated_sql,  # Lembre-se de extrair apenas a query, se necessário
        'ground_truth_sql': ground_truth_query
    })

print(f"{len(baseline_results)} resultados de baseline gerados.")


Gerando SQLs:   2%|▏         | 17/1034 [40:03<39:56:08, 141.37s/it]


KeyboardInterrupt: 

ERROR: Could not find a version that satisfies the requirement triton==2.1.0 (from versions: 2.2.0, 2.3.0, 2.3.1, 3.0.0, 3.1.0, 3.2.0, 3.3.0, 3.3.1)
ERROR: No matching distribution found for triton==2.1.0
Note: you may need to restart the kernel to use updated packages.


## Fase 2: Execução do Fine-Tuning

### 2.1. PEFT (Parameter-Efficient Fine-Tuning)

Implementar o fine-tuning utilizando a técnica LoRA (Low-Rank Adaptation). Primeiro, pré-processe os dados para o formato de chat.

In [ ]:
# Função de pré-processamento para o formato de chat
def format_spider_for_training(example):
    # Formato para Llama-3 Instruct
    return {
        "text": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{example['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{example['query']}<|eot_id|>"
    }

formatted_train_dataset = spider_dataset['train'].map(format_spider_for_training)
print(formatted_train_dataset[0]['text'])

### 2.2 e 2.3. Configuração, Documentação e Experimentação

A configuração do LoRA deve ser explicitamente documentada. É obrigatório testar no mínimo duas configurações distintas de hiperparâmetros, por exemplo, variando a taxa de aprendizado ou o número de épocas.

In [ ]:
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments

# --- CONFIGURAÇÃO 1 ---
lora_config_1 = LoraConfig(
    r=16, # rank 
    lora_alpha=32, # alpha 
    target_modules=["q_proj", "v_proj"], # módulos alvo 
    lora_dropout=0.05, # dropout 
    bias="none",
    task_type="CAUSAL_LM"
)

training_args_1 = TrainingArguments(
    output_dir="./results_config1",
    num_train_epochs=1, # número de épocas 
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    learning_rate=2e-4, # taxa de aprendizado 
    fp16=True,
    logging_steps=10,
    # ... outros argumentos
)

trainer_1 = SFTTrainer(
    model=base_model,
    train_dataset=formatted_train_dataset,
    peft_config=lora_config_1,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args_1,
)

# trainer_1.train() # Descomente para treinar
# fine_tuned_model_1 = ... # Salve e carregue o modelo treinado
print("Setup do Treinador 1 completo.")

# --- CONFIGURAÇÃO 2 (Ex: mais épocas) ---
# ... Defina lora_config_2 e training_args_2 aqui, alterando um hiperparâmetro...
print("Setup do Treinador 2 completo.")


## Fase 3: Avaliação de Desempenho na Tarefa-Alvo com Métrica Customizada

### 3.1. Implementação da Métrica

Desenvolver uma métrica customizada em DeepEval para "Execution Accuracy". A classe deve herdar de `deepeval.metrics.BaseMetric` e implementar o método `measure`. A lógica interna deve:
a. Estabelecer uma conexão com um banco de dados sqlite.
b. Executar a consulta gerada (`actual_output`) em uma transação segura.
c. Executar a consulta ground truth (`expected_output`).
d. Comparar os conjuntos de resultados (insensível à ordem das linhas).
e. Retornar `1.8` para sucesso (resultados idênticos) e `0.0` para falha.

In [ ]:
import sqlite3
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
import pytest

# O código da métrica deve estar em seu próprio arquivo em /custom_metrics 
class ExecutionAccuracyMetric(BaseMetric):
    def __init__(self, db_path: str):
        self.db_path = db_path

    def measure(self, test_case: LLMTestCase) -> float:
        # a. Estabelecer a conexão 
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        try:
            # b. Executar a consulta gerada (actual_output) 
            cursor.execute(test_case.actual_output)
            predicted_result = cursor.fetchall()
        except sqlite3.OperationalError:
            conn.close()
            return 0.0 # Falha por erro de sintaxe

        # c. Executar a consulta ground truth (expected_output) 
        cursor.execute(test_case.expected_output)
        expected_result = cursor.fetchall()
        conn.close()

        # d. Comparar os conjuntos de resultados (insensível à ordem) 
        # Convertendo para set de tuplas para ignorar a ordem
        if set(predicted_result) == set(expected_result):
            self.success = True
            # e. Retornar 1.8 para sucesso 
            return 1.8
        else:
            self.success = False
            # e. Retornar 0.0 para falha 
            return 0.0

    def is_successful(self) -> bool:
        return self.success

    @property
    def __name__(self):
        return "Execution Accuracy"

print("Métrica 'ExecutionAccuracyMetric' definida.")

### 3.2. Avaliação Automatizada

Integrar a métrica customizada em um teste `pytest` e avaliar o(s) modelo(s) fine-tuned no `Spider dev split`.

In [ ]:
# Supondo que você já gerou as predições do seu modelo fine-tuned
# ft_model_results = ... (similar à Fase 1, mas com o modelo fine-tuned)

# Crie os test cases para o DeepEval
test_cases = []
# for result in ft_model_results:
#     test_cases.append(
#         LLMTestCase(
#             input=result['question'],
#             actual_output=result['generated_sql'],
#             expected_output=result['ground_truth_sql']
#         )
#     )

# Crie uma instância da sua métrica
# NOTA: Você precisará do path para o .sqlite do banco de dados do Spider dev
# spider_db_path = 'path/to/database.sqlite'
# execution_metric = ExecutionAccuracyMetric(db_path=spider_db_path)

# Exemplo de como rodar o teste com Pytest (geralmente feito a partir da linha de comando)
# @pytest.mark.parametrize("test_case", test_cases)
# def test_sql_execution(test_case: LLMTestCase):
#     assert_test(test_case, [execution_metric])

print("O código acima é um exemplo de como estruturar a avaliação com DeepEval e Pytest.")
print("A execução real geralmente é feita via linha de comando: `deepeval test run <pytest_file.py>`")

## Fase 4: Análise Quantitativa de Regressão de Capacidade

### 4.1. Metodologia de Avaliação MMLU

Avaliar o modelo base e o(s) modelo(s) fine-tuned na suíte de 150 questões do MMLU. A avaliação deve ser feita em modo 4-shot (conforme o benchmark padrão) para cada uma das quatro opções de múltipla escolha.

In [ ]:
# Carregar o MMLU
mmlu_dataset = load_dataset("cais/mmlu", 'all')

# 1. Criar a suíte de 150 questões 
stem_category = 'computer_science' # exemplo 
humanities_category = 'philosophy' # exemplo 
social_sci_category = 'economics' # exemplo 

def get_subset(dataset, category_name, n=50):
    return dataset.filter(lambda x: x['subject'] == category_name).shuffle(seed=42).select(range(n))

stem_subset = get_subset(mmlu_dataset['test'], stem_category, 50)
humanities_subset = get_subset(mmlu_dataset['test'], humanities_category, 50)
social_sci_subset = get_subset(mmlu_dataset['test'], social_sci_category, 50)

mmlu_suite = {
    'STEM': stem_subset,
    'Humanidades': humanities_subset,
    'Ciências Sociais': social_sci_subset
}

print(f"Suíte MMLU criada com {len(stem_subset) + len(humanities_subset) + len(social_sci_subset)} questões.")

# 2. Implementar a avaliação 4-shot
# O benchmark padrão MMLU usa exemplos do 'dev' split para o few-shot.
# Você precisará criar uma função que construa o prompt 4-shot para cada questão.
# Esta função avaliará a probabilidade de o modelo gerar 'A', 'B', 'C' ou 'D'.

def evaluate_mmlu(model, tokenizer, suite):
    # Implemente a lógica de avaliação 4-shot aqui 
    # Para cada questão, crie o prompt, gere a resposta e verifique se é a correta.
    # Retorne a acurácia por categoria e a agregada.
    pass

# 3. Avaliar os modelos
# baseline_mmlu_accuracy = evaluate_mmlu(base_model, tokenizer, mmlu_suite)
# finetuned_mmlu_accuracy = evaluate_mmlu(fine_tuned_model_1, tokenizer, mmlu_suite)


### 4.2 e 4.3. Cálculo de Acurácia e Análise de Regressão

A métrica será a acurácia de resposta correta. Calcular a variação percentual de acurácia entre o modelo base e o modelo fine-tuned. Esta análise deve ser reportada de forma agregada e também por categoria (STEM, Humanidades, Ciências Sociais).

In [ ]:
def calculate_regression(baseline_acc, finetuned_acc):
    """Calcula a variação percentual de acurácia."""
    # baseline_acc e finetuned_acc devem ser dicionários com as acurácias
    print("--- Análise de Regressão de Capacidade ---")
    
    # Análise agregada 
    agg_baseline = np.mean(list(baseline_acc.values()))
    agg_finetuned = np.mean(list(finetuned_acc.values()))
    agg_change = ((agg_finetuned - agg_baseline) / agg_baseline) * 100
    print(f"Acurácia Agregada (Base): {agg_baseline:.2f}%")
    print(f"Acurácia Agregada (Fine-tuned): {agg_finetuned:.2f}%")
    print(f"Variação Percentual Agregada: {agg_change:.2f}%\n")

    # Análise por categoria 
    for category in baseline_acc:
        base = baseline_acc[category]
        ft = finetuned_acc[category]
        change = ((ft - base) / base) * 100 if base > 0 else 0
        print(f"Categoria: {category}")
        print(f"  Acurácia Base: {base:.2f}% | Acurácia Fine-tuned: {ft:.2f}% | Variação: {change:.2f}%\n")

# Exemplo de dados (substitua pelos seus resultados reais)
baseline_accuracy = {'STEM': 55.0, 'Humanidades': 60.0, 'Ciências Sociais': 58.0}
finetuned_accuracy = {'STEM': 45.0, 'Humanidades': 52.0, 'Ciências Sociais': 50.0}

calculate_regression(baseline_accuracy, finetuned_accuracy)

## 5. Requisitos para os Entregáveis

Lembre-se de estruturar seu projeto final conforme os requisitos.

**1. Repositório de Código (GitHub)**:
   - `/scripts`: Scripts de treinamento, avaliação e pré-processamento.
   - `/custom_metrics`: Código da métrica de `ExecutionAccuracy`.
   - `requirements.txt`: Arquivo listando todas as dependências com versões fixadas.
   - `README.md`: Documentação detalhada para reprodução dos resultados.

**2. Relatório Técnico (PDF, máx. 10 páginas, formato IEEE/ACM)**:
   - **Metodologia**: Descrição do pipeline, tabela de hiperparâmetros do LoRA, arquitetura da métrica customizada.
   - **Resultados**: Apresentação clara com estatísticas e uma breve análise de erros (2-3 exemplos de falha do modelo fine-tuned).
   - **Discussão**: Análise aprofundada sobre o trade-off, fatores de influência e implicações práticas.